In [1]:
from azureml.core import Workspace
ws = Workspace.from_config()

In [22]:
from azureml.core import Workspace

pat_token = "po5zbxsq"
ws = Workspace.from_config1()
ws.set_connection(name="connection-1", 
   category = "PythonFeed",
   target = "https://pkgs.dev.azure.com/The-Sam-Org1", 
   authType = "PAT", 
   value = pat_token)

AttributeError: type object 'Workspace' has no attribute 'from_config1'

In [20]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment(name="my-env-azure-pip-feed-And4")
cd = CondaDependencies()
# cd.add_pip_package("azureml-defaults")
cd.add_pip_package("pandas")
cd.set_pip_option("-i https://pkgs.dev.azure.com/The-Sam-Org/_packaging/pip_feed/pypi/simple")
cd.set_pip_option("--trusted-host pypi.org")
env.python.conda_dependencies=cd

In [21]:
# Register the environment
env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220113.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "my-env-azure-pip-feed-And4",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "

In [19]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', ], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Dataset already registered.


In [20]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aml-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [37]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory='training-example',
                      script='train.py',
                      compute_target=cluster_name,
                      environment=my_env)

In [38]:
# submit the experiment
from azureml.core import Experiment
experiment_name = 'prebuilt-docker-image-test-1'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=src)


In [39]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…